In [129]:
import numpy as np
import pandas as pd

In [130]:
df = pd.read_csv('listings.csv.gz')

In [131]:
df2 = df.copy()

In [132]:
df_numeric = df.select_dtypes(include=['float64'])

df_string = df.select_dtypes(include=['object'])

df_int = df.select_dtypes(include=['int'])

In [133]:
df_string[potential_important_string].describe()

,host_since,host_location,host_response_rate,host_acceptance_rate,host_is_superhost,host_identity_verified,neighbourhood,neighbourhood_cleansed,property_type,room_type,bathrooms_text,instant_bookable,last_review
count,18228,18199,3937,7796,18228,18228,12023,18291,18291,18291,18269,18291,15902
unique,3268,534,50,89,2,2,67,22,65,4,27,2,1833
top,2016-04-05,"Amsterdam, North Holland, Netherlands",100%,100%,f,t,"Amsterdam, Noord-Holland, Netherlands",De Baarsjes - Oud-West,Entire apartment,Entire home/apt,1 bath,f,2020-01-02
freq,88,10046,2794,3638,15849,11918,9534,3044,11393,14283,10677,13914,259


In [134]:
drop_list = ['neighbourhood_group_cleansed', 'bathrooms', 'calendar_updated', 'license', 'latitude', 'longitude', 'id',\
             'scrape_id', 'host_id']

In [135]:
df2.drop(columns = drop_list, axis = 1, inplace = True)
df2.shape

(18291, 65)

In [136]:
potential_important_string = ['host_since', 'host_location', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',\
                             'host_identity_verified', 'neighbourhood', 'neighbourhood_cleansed',\
                             'property_type', 'room_type', 'bathrooms_text', 'instant_bookable', 'last_review']

In [137]:
df2.dropna(subset = ['host_since', 'host_is_superhost', 'host_identity_verified'], inplace = True)

In [138]:
# cleaning and engineering host_since
df2['host_since'] = pd.to_datetime(df2['host_since'])
df2['months_active'] = (dt.datetime.today() - df2['host_since'])/np.timedelta64(1, 'M')
df2['months_active'] = df2['months_active'].astype(int)

In [139]:
df2['host_location'].value_counts()
# too many entries to clean, ignore for now

Amsterdam, North Holland, Netherlands           10046
Amsterdam, North Holland, The Netherlands        3515
NL                                               1549
Amsterdam, Noord-Holland, The Netherlands        1153
Netherlands                                       520
                                                ...  
Rio de Janeiro, Rio de Janeiro, Brazil              1
Pretoria, Gauteng, South Africa                     1
Beverwijk, North Holland, The Netherlands           1
Newcastle upon Tyne, England, United Kingdom        1
De Goorn, North Holland, The Netherlands            1
Name: host_location, Length: 534, dtype: int64

In [140]:
df2['host_response_rate'] = df2['host_response_rate'].str.replace('%','')
df2['host_response_rate'] = pd.to_numeric(df2['host_response_rate'])
print(df2['host_response_rate'].describe())
# Given most of the values being missing, we will replace missing values with mean
df2['host_response_rate'].fillna(df2['host_response_rate'].mean(), inplace = True)
# print(df2['host_response_rate'].describe())

count    3937.000000
mean       86.042418
std        28.633557
min         0.000000
25%        89.000000
50%       100.000000
75%       100.000000
max       100.000000
Name: host_response_rate, dtype: float64


In [141]:
df2['host_acceptance_rate'] = df2['host_acceptance_rate'].str.replace('%','')
df2['host_acceptance_rate'] = pd.to_numeric(df2['host_acceptance_rate'])
print(df2['host_acceptance_rate'].describe())
# Given most of the values being missing, we will replace missing values with mean
df2['host_acceptance_rate'].fillna(df2['host_acceptance_rate'].mean(), inplace = True)
print(df2['host_acceptance_rate'].describe())

count    7796.000000
mean       77.234864
std        33.575089
min         0.000000
25%        63.000000
50%        98.000000
75%       100.000000
max       100.000000
Name: host_acceptance_rate, dtype: float64
count    18228.000000
mean        77.234864
std         21.956729
min          0.000000
25%         77.234864
50%         77.234864
75%         93.000000
max        100.000000
Name: host_acceptance_rate, dtype: float64


In [142]:
print(df2['host_is_superhost'].value_counts())
# only true false values, convert them to 0 and 1 categorical
df2['host_is_superhost'].replace({'t': 1, 'f': 0}, inplace = True)
df2['host_is_superhost'].value_counts()

f    15849
t     2379
Name: host_is_superhost, dtype: int64


0    15849
1     2379
Name: host_is_superhost, dtype: int64

In [145]:
print(df2['host_identity_verified'].value_counts())
# only true false values, convert them to 0 and 1 categorical
df2['host_identity_verified'].replace({'t': 1, 'f': 0}, inplace = True)
df2['host_identity_verified'].value_counts()

t    11918
f     6310
Name: host_identity_verified, dtype: int64


1    11918
0     6310
Name: host_identity_verified, dtype: int64

In [146]:
df2['neighbourhood'].value_counts()
# too many values, ignore for now

Amsterdam, Noord-Holland, Netherlands             9487
Amsterdam, North Holland, Netherlands             1483
Amsterdam, NH, Netherlands                         697
Amsterdam, Netherlands                             110
Amsterdam-Zuidoost, Noord-Holland, Netherlands      57
                                                  ... 
Amsterdam, boord Holland, Netherlands                1
1015 SN Amsterdam, Noord-Holland, Netherlands        1
Halfweg, NH, Netherlands                             1
Amsterdam, AH, Netherlands                           1
Duivendrecht, Noord-Holland, Netherlands             1
Name: neighbourhood, Length: 67, dtype: int64

In [147]:
df2['neighbourhood_cleansed'].value_counts()
# manageable number of values, will do one hot encoding

De Baarsjes - Oud-West                    3029
De Pijp - Rivierenbuurt                   2262
Centrum-West                              1994
Centrum-Oost                              1554
Westerpark                                1392
Zuid                                      1306
Oud-Oost                                  1177
Bos en Lommer                             1042
Oostelijk Havengebied - Indische Buurt     869
Oud-Noord                                  586
Watergraafsmeer                            525
IJburg - Zeeburgereiland                   423
Slotervaart                                410
Noord-West                                 369
Noord-Oost                                 254
Buitenveldert - Zuidas                     252
Geuzenveld - Slotermeer                    214
De Aker - Nieuw Sloten                     126
Osdorp                                     124
Gaasperdam - Driemond                      121
Bijlmer-Centrum                            103
Bijlmer-Oost 